In [1]:
!python -V

Python 3.9.19


In [ ]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1718809930065, experiment_id='1', last_update_time=1718809930065, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')

In [10]:
len(df_train), len(df_val)

(54373, 51497)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.9947992164797

In [19]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [25]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [26]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:6.83465                           
[1]	validation-rmse:5.93499                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:17:42] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:5.61749                           
[3]	validation-rmse:5.50422                           
[4]	validation-rmse:5.45300                           
[5]	validation-rmse:5.43174                           
[6]	validation-rmse:5.42036                           
[7]	validation-rmse:5.41480                           
[8]	validation-rmse:5.41183                           
[9]	validation-rmse:5.40905                           
[10]	validation-rmse:5.40652                          
[11]	validation-rmse:5.40190                          
[12]	validation-rmse:5.39667                          
[13]	validation-rmse:5.39464                          
[14]	validation-rmse:5.39119                          
[15]	validation-rmse:5.39131                          
[16]	validation-rmse:5.38688                          
[17]	validation-rmse:5.38453                          
[18]	validation-rmse:5.38056                          
[19]	validation-rmse:5.37865                          
[20]	valid

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:17:59] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.73920                                                    
[1]	validation-rmse:5.43849                                                    
[2]	validation-rmse:5.41168                                                    
[3]	validation-rmse:5.40225                                                    
[4]	validation-rmse:5.39862                                                    
[5]	validation-rmse:5.39912                                                    
[6]	validation-rmse:5.40038                                                    
[7]	validation-rmse:5.39660                                                    
[8]	validation-rmse:5.39163                                                    
[9]	validation-rmse:5.39122                                                    
[10]	validation-rmse:5.39059                                                   
[11]	validation-rmse:5.37241                                                   
[12]	validation-rmse:5.37322            

...

[0]	validation-rmse:7.58253                                                    
[1]	validation-rmse:6.62844                                                    
[2]	validation-rmse:6.05661                                                    
[3]	validation-rmse:5.72726                                                    
[4]	validation-rmse:5.54158                                                    
[5]	validation-rmse:5.43721                                                    
[6]	validation-rmse:5.37929                                                    
[7]	validation-rmse:5.34581                                                    
[8]	validation-rmse:5.32602                                                    
[9]	validation-rmse:5.30950                                                    
[10]	validation-rmse:5.29936                                                   
[11]	validation-rmse:5.28959                                                   
[12]	validation-rmse:5.28375            

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:24:45] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.53679                                                     
[1]	validation-rmse:5.62967                                                     
[2]	validation-rmse:5.34045                                                     
[3]	validation-rmse:5.25465                                                     
[4]	validation-rmse:5.21888                                                     
[5]	validation-rmse:5.20148                                                     
[6]	validation-rmse:5.18884                                                     
[7]	validation-rmse:5.18643                                                     
[8]	validation-rmse:5.18648                                                     
[9]	validation-rmse:5.18526                                                     
[10]	validation-rmse:5.18607                                                    
[11]	validation-rmse:5.18518                                                    
[12]	validation-rmse:5.18530

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:25:03] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.53298                                                     
[1]	validation-rmse:8.03126                                                     
[2]	validation-rmse:7.59819                                                     
[3]	validation-rmse:7.22587                                                     
[4]	validation-rmse:6.90582                                                     
[5]	validation-rmse:6.63399                                                     
[6]	validation-rmse:6.40283                                                     
[7]	validation-rmse:6.20795                                                     
[8]	validation-rmse:6.04423                                                     
[9]	validation-rmse:5.90705                                                     
 22%|██▏       | 11/50 [07:34<27:10, 41.81s/trial, best loss: 5.138968589677634]

In [20]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:33:46] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.55121
[1]	validation-rmse:8.06400
[2]	validation-rmse:7.64347
[3]	validation-rmse:7.28077
[4]	validation-rmse:6.96868
[5]	validation-rmse:6.70403
[6]	validation-rmse:6.47713
[7]	validation-rmse:6.28420
[8]	validation-rmse:6.12122
[9]	validation-rmse:5.98447
[10]	validation-rmse:5.87103
[11]	validation-rmse:5.77302
[12]	validation-rmse:5.68988
[13]	validation-rmse:5.62238
[14]	validation-rmse:5.56459
[15]	validation-rmse:5.51506
[16]	validation-rmse:5.47349
[17]	validation-rmse:5.43747
[18]	validation-rmse:5.40878
[19]	validation-rmse:5.38302
[20]	validation-rmse:5.36147
[21]	validation-rmse:5.34029
[22]	validation-rmse:5.32216
[23]	validation-rmse:5.30792
[24]	validation-rmse:5.29485
[25]	validation-rmse:5.28297
[26]	validation-rmse:5.27297
[27]	validation-rmse:5.26478
[28]	validation-rmse:5.25749
[29]	validation-rmse:5.25086
[30]	validation-rmse:5.24386
[31]	validation-rmse:5.23724
[32]	validation-rmse:5.23377
[33]	validation-rmse:5.23046
[34]	validation-rmse:5.2

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:34:38] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
